In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

### Load your data

In [2]:
loader = TextLoader('/Users/pei-jrweng/Documents/my_code_projects/telopenbot/data/meditation.txt')

In [3]:
data = loader.load()

In [4]:
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your document')

You have 1 document(s) in your data
There are 187584 characters in your document


### Chunk your data up into smaller documents

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [6]:
print (f'Now you have {len(texts)} documents')

Now you have 191 documents


### Create embeddings of your documents to get ready for semantic search

In [7]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone
import os
from dotenv import load_dotenv

load_dotenv()

/Users/pei-jrweng/Documents/my_code_projects/telopenbot/venv/lib/python3.9/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


True

In [8]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV = 'us-east1-gcp'

In [9]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [10]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "meditation"

In [11]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [12]:
query = "How can I overcome distractions while meditating?"
docs = docsearch.similarity_search(query, include_metadata=True, k=3)
docs

[Document(page_content='這個日常作息聽起來或許不那麼累人，畢竟除了禪修，其他事情都\n\n跟我們一般認定的工作毫無瓜葛，但剛開始前幾天還挺折磨人的。你\n\n有試過盤腿坐在蒲團上，並聚焦於自己的呼吸上嗎？這可不是在野\n\n餐，尤其當你像我一樣不善於聚焦在自己的呼吸時，更是如此。在靜\n\n坐的前幾日，我在整整四十五分鐘的課程中，甚至無法連續專注在自\n\n己的呼吸上達到十次。我之所以知道，是因為我在計算！只要數到第\n\n三或第四次，我的心神就開始遊蕩，然後不知道自己數到哪裡了，屢\n\n試不爽。有時候其實是我仍在數數，但腦子已經想到別的事情，沒有\n\n以意識去感受呼吸。\n\n每次發生這種事，我就會生自己的氣，而且在禪七的前幾天，我變\n\n得越來越生氣──而這對我並沒有幫助。接著，我的怒氣開始擴及到那\n\n些做得比我好的人身上，大概有八十人這麼多吧──也就是在場的所有\n\n人。想像一下，要與八十個做得比你好的人，待在一起一整週！那些\n\n人都「成功」了，而你卻「一敗塗地」。\n\n我的重大突破\n\n到了第五天早上，我終於有了重大突破。早餐之後，我喝了太多自\n\n己帶來的即溶咖啡，導致接著要開始靜坐時，出現咖啡因攝取過多的\n\n典型症狀：下顎緊繃到令我不適，覺得自己在咬牙切齒，不斷干擾我\n\n保持專注。與這種干擾搏鬥了一會兒，最後我索性放棄搏鬥，把注意\n\n力轉移到緊繃的下顎。或者，我並不是真的「轉移」了注意力，而是\n\n「擴大」了注意力：我繼續意識到自己在呼吸，但是退讓到後方，讓\n\n惱人的下顎緊繃感移動到舞臺中央。\n\n順帶一提，這種重新調整注意力的做法是完全沒有問題的。在典型\n\n的正念靜坐教導中，聚焦於呼吸的關鍵在於：並非只聚焦在呼吸，還\n\n要穩定心緒，讓它從一般所關注的事情中釋放出來。如此一來，你便\n\n可在清明、從容、緩和的方式中，觀察到當下發生的事情。而所謂\n\n「當下發生的事情」，當然也包括那個瞬間發生在你內心的事，那些\n\n從內部湧出的感受：悲傷、焦慮、煩惱、放鬆、喜悅等。而你試著要\n\n從不同於以往的觀點來經歷這些感受，既不緊抓著這些美好感受不\n\n放，也不逃離那些不好的感受，而是直接了當地去經歷、去觀察。這\n\n種視角的更動，將會徹底、持久地改變你和自身感受之間的關係。要\n

### Query those docs to get your answer back

In [13]:
from langchain.chat_models import ChatOpenAI
# from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [14]:
llm = ChatOpenAI(temperature=1, openai_api_key=OPENAI_API_KEY, model_name="gpt-4")
chain = load_qa_chain(llm, chain_type="stuff")

In [15]:
query = "How can I overcome distractions while meditating?"
docs = docsearch.similarity_search(query, include_metadata=True)

In [16]:
chain.run(input_documents=docs, question=query)

"To overcome distractions while meditating, try the following techniques:\n\n1. Set a clear intention: Before starting your meditation, set a clear intention to maintain focus and be present.\n\n2. Choose a suitable environment: Find a quiet and comfortable space, free from external distractions or noise, to meditate.\n\n3. Start with short sessions: Begin with shorter meditation sessions to help build your focus gradually. As you get better at handling distractions, you can increase the duration of your sessions.\n\n4. Focus on your breath: Concentrate on your breathing as it provides a natural point of focus. Observe the inhalation and exhalation process, and try to maintain awareness of your breath throughout your meditation.\n\n5. Be gentle with yourself: When your mind wanders or gets distracted, gently bring your focus back to the breath. Don't judge yourself or get frustrated; remember that it is natural for the mind to wander.\n\n6. Acknowledge distractions: When distractions a